In [1]:
import pandas as pd
import numpy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

from sklearn.decomposition import PCA
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

import time
from datetime import datetime 

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

start_time = datetime.now() 

In [2]:
test = pd.read_csv("C:\\MyFoulder\\SCE\שנה ג\\סמסטר ב\\מבוא ללמידה עמוקה\\3\\ex3_test_data.csv")
train = pd.read_csv("C:\\MyFoulder\\SCE\שנה ג\\סמסטר ב\\מבוא ללמידה עמוקה\\3\\ex3_train_data.csv")
train

,id,sentence,label
0,0,"considering this , we assumed that it is a ver...",0
1,1,"according to mdl , the best probability model ...",1
2,2,response complexity : there is a reward and a ...,1
3,3,we therefore divide by this score to assure th...,0
4,4,here is an overview of the chronology of events .,0
...,...,...,...
1495,1495,2.2 the input to nlg from the dialogue manager...,0
1496,1496,"for a set of annotated examples , we used deci...",0
1497,1497,"in the present context , however , the limitat...",0
1498,1498,the individual fb1 scores for nps were 92 .19 ...,0


In [3]:
test

,id,sentence
0,1500,there is a rich variety of colour descriptions...
1,1501,es99 define the following *i predicates ( ecke...
2,1502,domain knowledge management in general involve...
3,1503,2this corpus is collected and annotated for th...
4,1504,other realization and sentence planning tin { ...
...,...,...
679,2179,"for example , the question `` what is the name..."
680,2180,"instead , knn performs online scoring to find ..."
681,2181,evius is a component of a multilingual ie syst...
682,2182,the base model defines the distance between a ...


In [4]:
train['sentence'] = train['sentence'].str.replace('[^\w\s]','')
train['sentence'] = train['sentence'].str.replace('[^a-zA-z ]','')

test['sentence'] = test['sentence'].str.replace('[^\w\s]','')
test['sentence'] = test['sentence'].str.replace('[^a-zA-z ]','')


stop_words = set(stopwords.words('english'))

for ID in  range(len(train['sentence'])):
    querywords = train['sentence'][ID].split()
    resultwords  = [word for word in querywords if word not in stop_words]
    train['sentence'][ID] = ' '.join(resultwords)
    
for ID in  range(len(test['sentence'])):
    querywords = test['sentence'][ID].split()
    resultwords  = [word for word in querywords if word not in stop_words]
    test['sentence'][ID] = ' '.join(resultwords)

<ipython-input-4-05d3efd386d9>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['sentence'][ID] = ' '.join(resultwords)
<ipython-input-4-05d3efd386d9>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sentence'][ID] = ' '.join(resultwords)


In [5]:
train

,id,sentence,label
0,0,considering assumed promising domain experimen...,0
1,1,according mdl best probability model given set...,1
2,2,response complexity reward punishment associat...,1
3,3,therefore divide score assure perfect sentence...,0
4,4,overview chronology events,0
...,...,...,...
1495,1495,input nlg dialogue manager frame attributevalu...,0
1496,1496,set annotated examples used decisiontree tools...,0
1497,1497,present context however limitation inessential...,0
1498,1498,individual fb scores nps vps pps,0


In [6]:
test

,id,sentence
0,1500,rich variety colour descriptions including bas...
1,1501,es define following predicates eckert strube b...
2,1502,domain knowledge management general involves t...
3,1503,corpus collected annotated gnome project poesi...
4,1504,realization sentence planning tin kswhich need...
...,...,...
679,2179,example question name creek
680,2180,instead knn performs online scoring find train...
681,2181,evius component multilingual ie system mturbio...
682,2182,base model defines distance test item memory i...


In [7]:
train.isnull().sum()

id          0
sentence    0
label       0
dtype: int64

In [8]:
test.isnull().sum()

id          0
sentence    0
dtype: int64

In [9]:
train.dropna(subset = ['sentence'], inplace=True)
test.dropna(subset = ['sentence'], inplace=True)

In [10]:
X = train.drop(['id', 'label'], axis=1)
X

,sentence
0,considering assumed promising domain experimen...
1,according mdl best probability model given set...
2,response complexity reward punishment associat...
3,therefore divide score assure perfect sentence...
4,overview chronology events
...,...
1495,input nlg dialogue manager frame attributevalu...
1496,set annotated examples used decisiontree tools...
1497,present context however limitation inessential...
1498,individual fb scores nps vps pps


In [11]:
y = train['label']
y

0       0
1       1
2       1
3       0
4       0
       ..
1495    0
1496    0
1497    0
1498    0
1499    0
Name: label, Length: 1500, dtype: int64

In [12]:
j = 0
for i in range(len(y)):
    if y[i]==1:
        j+=1
print('Num of 1\'s is: ', j)

Num of 1's is:  499


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [14]:
X_train
#X_test
#y_train
#y_test

,sentence
107,section describes two experiments briefly
631,sometimes
1209,allied corpussimilarity corpushomogeneity
797,since referent identifiable definite descripti...
1003,uce filtering text categorization task
...,...
1190,englishgerman translation like englishspanish ...
125,let training documents number training documen...
898,course satisfied metric best advantage generat...
592,mt systems examined light outputs translation ...


In [15]:
sentences = [w.split() for w in X_test['sentence']]
print(len(sentences))
model = Word2Vec(sentences, min_count=1, vector_size=300)
print(model)

words = list(model.wv.index_to_key)
len(words)

zer = numpy.array([0.0 for x in range(300)])

sen = list(X_test['sentence'])
vecs = []
vecs2 = []
for s in sen:
    if len(s.split())<40:
        for w in s.split():
            vecs.append(numpy.array(model.wv.get_vector(w, norm=True)))
        if len(s.split()) == 0:
            vecs.append(zer)
        vecs2.append(numpy.array(vecs))
        vecs.clear()
    else:
        vecs2.append(numpy.array([zer for x in range(30)]))
    
for i in range(len(vecs2)):
    while len(vecs2[i])<40:
        vecs2[i] = numpy.vstack([vecs2[i], zer])

X_test_vecs = vecs2

450
Word2Vec(vocab=2657, vector_size=300, alpha=0.025)


In [16]:
numpy.array(X_test_vecs).shape

(450, 40, 300)

In [17]:
sentences = [w.split() for w in X_train['sentence']]
print(len(sentences))
model = Word2Vec(sentences, min_count=1, vector_size=300)
print(model)

words = list(model.wv.index_to_key)
len(words)

zer = numpy.array([0.0 for x in range(300)])

sen = list(X_train['sentence'])
vecs = []
vecs2 = []
for s in sen:
    if len(s.split())<40:
        for w in s.split():
            vecs.append(numpy.array(model.wv.get_vector(w, norm=True)))
        if len(s.split()) == 0:
            vecs.append(zer)
        vecs2.append(numpy.array(vecs))
        vecs.clear()
    else:
        vecs2.append(numpy.array([zer for x in range(30)]))
    
for i in range(len(vecs2)):
    while len(vecs2[i])<40:
        vecs2[i] = numpy.vstack([vecs2[i], zer])

X_train_vecs = vecs2

1050
Word2Vec(vocab=4411, vector_size=300, alpha=0.025)


In [18]:
numpy.array(vecs2).shape

(1050, 40, 300)

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Conv1D, LeakyReLU, Dropout

model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(40,300)))
#model.add(LeakyReLU(alpha=0.2))
#model.add(Dropout(0.4))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(numpy.array(X_train_vecs), y_train, validation_data=(numpy.array(X_test_vecs), y_test), epochs=5)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 38, 32)            28832     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 36, 64)            6208      
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2305      
Total params: 37,345
Trainable params: 37,345
Non-trainable params: 0
_________________________________________________________________


NameError: name 'numpy' is not defined

In [88]:
sentences = [w.split() for w in test['sentence']]
print(len(sentences))
model2 = Word2Vec(sentences, min_count=1, vector_size=300)
print(model2)

words = list(model2.wv.index_to_key)
len(words)

zer = numpy.array([0.0 for x in range(300)])

sen = list(test['sentence'])
vecs = []
vecs2 = []
for s in sen:
    if len(s.split())<40:
        for w in s.split():
            vecs.append(numpy.array(model2.wv.get_vector(w, norm=True)))
        if len(s.split()) == 0:
            vecs.append(zer)
        vecs2.append(numpy.array(vecs))
        vecs.clear()
    else:
        vecs2.append(numpy.array([zer for x in range(30)]))
    
for i in range(len(vecs2)):
    while len(vecs2[i])<40:
        vecs2[i] = numpy.vstack([vecs2[i], zer])

f_vecs = vecs2

684
Word2Vec(vocab=3569, vector_size=300, alpha=0.025)


In [94]:
test_pred = (model.predict_classes(numpy.array(f_vecs)) > 0.5).astype("int32")

In [91]:
df3 = pd.read_csv("C:\\MyFoulder\\SCE\שנה ג\\סמסטר ב\\מבוא ללמידה עמוקה\\3\\ex3_sampleSubmission.csv")
df3['label'] = test_pred
    
df3.to_csv('Project3.12.csv', index = False)

In [33]:
print('Time elapsed in (hh:mm:ss.ms): "{}"'.format(datetime.now() - start_time))

Time elapsed in (hh:mm:ss.ms): "0:00:03.583826"
